In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset, random_split
from PIL import Image
from tqdm import tqdm
import numpy as np
from sklearn.metrics import r2_score
import numpy as np

In [2]:
# Split train into train and validate again
new_train_csv = pd.read_csv('data/enhanced_train.csv')
new_test_csv = pd.read_csv('data/enhanced_test.csv')
output_mean = [1.03624107e+00, 1.48317376e+02, 1.97016450e+04, 3.48191181e+03, 1.51120666e+01, 3.99120598e+05]
output_std = [1.37329381e-01, 6.91740145e+00, 4.31037489e+00, 6.70979751e+01, 5.93192463e-01, 2.25494269e+03]
output_mean = np.array(output_mean)
output_std = np.array(output_std)

IDs_train = new_train_csv.iloc[:, 0].values
X_train = new_train_csv.iloc[:, 1:-6].values
Y_train = new_train_csv.iloc[:, -6:].values

IDs_test = new_test_csv.iloc[:, 0].values
X_test = new_test_csv.iloc[:, 1:].values

# Answer to life, universe, and everything for goodluck
X_train, X_val, Y_train, Y_val, IDs_train, IDs_val = train_test_split(
    X_train, Y_train, IDs_train, test_size=1/11, random_state=42
)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
Y_val_tensor = torch.tensor(Y_val, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [3]:
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.layer1 = nn.Linear(input_dim, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, output_dim)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = self.output(x)
        return x


In [4]:
input_dim = X_train_tensor.shape[1]
output_dim = Y_train_tensor.shape[1]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel(input_dim, output_dim).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.01)

In [5]:
def R2eval(model, X_val_tensor, Y_val_tensor):
    model.eval()
    with torch.no_grad():
        X_val_tensor = X_val_tensor.to(device)
        Y_val_tensor = Y_val_tensor.to(device)
        val_predictions = model(X_val_tensor).cpu().numpy()
        val_predictions = val_predictions * output_std + output_mean
        val_predictions_tensor = torch.tensor(val_predictions, dtype=torch.float32)
        val_predictions_tensor = val_predictions_tensor.to(device)
        val_r2 = r2_score(Y_val, val_predictions)
    return val_r2

In [6]:
epochs = 100
best_model = model
bestR2 = float('-inf')

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch_X, batch_Y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        batch_Y = (batch_Y - output_mean) / output_std
        batch_X, batch_Y = batch_X.to(device).float(), batch_Y.to(device).float()
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_Y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    currentR2 = R2eval(model, X_val_tensor, Y_val_tensor)
    print(f"Current Estimate R2 is {currentR2}")
    if (currentR2 > bestR2):
        best_model = model
        bestR2 = currentR2
        print("Saved model")
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(train_loader)}")

Epoch 1/100: 100%|██████████| 1232/1232 [00:01<00:00, 712.53it/s]


Current Estimate R2 is 0.4430721541059537
Saved model
Epoch 1/100, Loss: 0.619124738838185


Epoch 2/100: 100%|██████████| 1232/1232 [00:01<00:00, 742.55it/s]


Current Estimate R2 is 0.4453653059801737
Saved model
Epoch 2/100, Loss: 0.5788445248474161


Epoch 3/100: 100%|██████████| 1232/1232 [00:01<00:00, 718.18it/s]


...

Current Estimate R2 is 0.45788339328730765
Epoch 98/100, Loss: 0.5658043890272255


Epoch 99/100: 100%|██████████| 1232/1232 [00:01<00:00, 776.51it/s]


Current Estimate R2 is 0.4613438973172656
Epoch 99/100, Loss: 0.566162646179656


Epoch 100/100: 100%|██████████| 1232/1232 [00:01<00:00, 768.08it/s]

Current Estimate R2 is 0.45942402012931344
Epoch 100/100, Loss: 0.5658085811883211


In [8]:
X_test_tensor = X_test_tensor.to(device)

with torch.no_grad():
    test_predictions = best_model(X_test_tensor).cpu().numpy()
    test_predictions = test_predictions * output_std + output_mean

predictions_df = pd.DataFrame(test_predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
predictions_df.insert(0, 'id', IDs_test)

# Save the predictions to a CSV file
predictions_df.to_csv('predictions.csv', index=False)